# Hermetic Wizard Malware
> Analysis of the Hermetic Wizard malware used to spread Hermetic Wiper in the Ukrainian cyber attacks

- toc: true 
- badges: true
- categories: [hermetic,hermetic wizard,spreader,malware,APT]

## Overview

In addition to the [Hermetic Wiper](https://www.sentinelone.com/labs/hermetic-wiper-ukraine-under-attack/) malware there were at least three other components used to deploy the wiper bundled into a single binary and referred to as [HermeticWizard](https://www.welivesecurity.com/2022/03/01/isaacwiper-hermeticwizard-wiper-worm-targeting-ukraine/). 

HermeticWizard is started using the command line `regsvr32.exe /s /i <path>`.


### Samples
HermeticWizard: `a259e9b0acf375a8bef8dbc27a8a1996ee02a56889cba07ef58c49185ab033ec` 
[MalwareBazaar Sample](https://bazaar.abuse.ch/browse/tag/hermeticwizard/)

In the resources of HermeticWizard are two spreader components, a WMI spreader, and an SMB spreader.

#### WMI Spreader (exec_x32.dll)
Samples: `2d29f9ca1d9089ba0399661bb34ba2fd8aba117f04678cd71856d5894aa7150b` 

#### SMB Spreader (romance.dll)
Sample: `5a300f72e221a228e3a36a043bef878b570529a7abc15559513ea07ae280bb48`



## WMI Spreader Analysis

The WMI spreader is a stand-alone untility that is used to copy files to remote shares and execute them. The spreader is excuted via HermeticWizard using the following example command line.

> `rundll32 <spreader path> #1 -s <path to HermeticWizard> – i <target IP>`

The spreader accepts the following command line arguments: `-i`,`-h`,`-s`,`-a`,`-c`

- **`s`** path to the file to copy
- **`i`** remote host IP
- **`h`** optional remote share path (*not confirmed)
- **`a`** optional brute force share password (*not confirmed)
- **`c`** optional brute force share domain username password list (*not confirmed)

## Network Service Account 
The spreader checks to see if it has local admin priviledges. If it has local admin it attempts to impersonate the `Network Service` service account. 

[Stack Overflow reference](https://stackoverflow.com/questions/510170/the-difference-between-the-local-system-account-and-the-network-service-acco)

> NetworkService account
>
> Limited service account that is meant to run standard privileged services. This account is > far more limited than Local System (or even Administrator) but still has the right to access the network as the machine (see caveat above).
> 
> - NT AUTHORITY\NetworkService
> - the account has no password (any password information you provide is ignored)
> - HKCU represents the NetworkService user account
> - has minimal privileges on the local computer
> - presents the computer's credentials to remote servers
> - **SID: S-1-5-20**
> - has its own profile under the HKEY_USERS registry key (HKEY_USERSS-1-5-20)
> - If trying to schedule a task using it, enter NETWORK SERVICE into the Select User or Group > dialog 

## Share file copy 
The spreader attempts to authenticate to the ADMIN&#36; remote share. If a password list is  supplied on the command line the share authentication is brute forced. Once authenticated the files are copied to the share.

## WMI COM Interface
For programatic access to WMI the wiper uses COM. 

`CLSID_WbemLocator {4590f811-1d3a-11d0-891f-00aa004b2e24}`

`IID_IWbemLocator  {dc12a687-737f-11cf-884d-00aa004b2e24}`

Big thanks to Mike Bailey for his [COM Presentation](https://github.com/strictlymike/presentations/blob/master/2020/2020.02.08_BSidesHuntsville/2020.02.08_Gophe_v1.5.pdf). He also has a nice (free) video on [Pluralsight](https://app.pluralsight.com/course-player?clipId=f1599111-8837-4fb2-a74f-83668cfa9f16).

IDA has the vtable for `IWbemLocator` as a struct so we can just directly apply it.

### Creating Processes Remotely using WMI

[MSDN Reference](https://docs.microsoft.com/en-us/windows/win32/wmisdk/creating-processes-remotely)

[Similar example code from GitHub](https://github.com/Yaxser/CobaltStrike-BOF/blob/e250fab812c0d95f6429c8fe8f61b9ea9e930f3c/WMI%20Lateral%20Movement/WMI-ProcessCreate.cpp)

Connection to remote host via [WMI](https://docs.microsoft.com/en-us/windows/win32/wmisdk/connecting-to-wmi-on-a-remote-computer) using `\\<target_ip>\root\cimv2`


[impersonate another user in CoCreateInstanceEx using COAUTHIDENTITY](https://stackoverflow.com/questions/10987142/how-do-i-properly-impersonate-another-user-in-cocreateinstanceex-using-coauthide) with [COAUTHIDENTITY](https://docs.microsoft.com/en-us/windows/win32/api/wtypesbase/ns-wtypesbase-coauthidentity)

Use WMI to access remote hosts
- Method `Win32_Process` `Create` `CommandLine`
- Command executed via WMI: 

`C:\\Windows\\system32\\cmd.exe /c start C:\\Windows\\system32\\regsvr32.exe /s /i C:\\Windows\\%s.dll`

This is used to launch the HermeticWizard binary that has been copied to the remote host.
